# This notebook:
1. Imports goalie_name_list csv  David's goalkeeper's data  & cleans & merges them into a single dataframe, goalie_hist_model_ready
2. Also created another datframe for csv export that has position id, first_name, second_name, all individual ids

In [34]:
# Dependencies
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.inspection import PartialDependenceDisplay

import matplotlib.pyplot as plt

In [35]:
#import master player data

goalie_name_list = pd.read_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/DFQ_scratch_work/final_work/goalie_name_list.csv")
goalie_name_list.columns
goalie_name_list["element_code"] = goalie_name_list["code"]
goalie_name_list.head()


,id,first_name,second_name,team,element_type,code,element_code
0,5,Karl,Hein,1,1,463748,463748
1,15,David,Raya Martin,1,1,154561,154561
2,73,Norberto,Murara Neto,1,1,69752,69752
3,639,Tommy,Setford,1,1,551221,551221
4,39,Joe,Gauci,2,1,462492,462492


In [75]:


ahistorical_goalie_data = pd.read_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/data/processed/from_david_goalkeepers_data.csv")
pd.set_option("display.max.columns", None)
ahistorical_goalie_data.columns
ahistorical_goalie_data['season'] = ahistorical_goalie_data['season_name'].str[:4].astype(int)

bhistorical_goalie_data = ahistorical_goalie_data.drop(columns='season_name')
historical_goalie_data = bhistorical_goalie_data[['season', 'element_code', 'start_cost', 'end_cost', 'total_points', 'minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded']]

historical_goalie_data.head()



,season,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,2007,40383,40,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
1,2008,40383,40,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
2,2011,40383,45,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
3,2014,40383,50,53,118,2623,0,0,13,21,0,0,0,1,0,61,0,442,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
4,2015,40383,50,49,68,1620,0,0,6,17,0,0,0,0,0,52,3,310,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0


In [82]:
#This dataframe combines the list of goalie names with 
# David's historical performance csv/dataframe. 


master_historical_goalie_data = pd.merge(goalie_name_list, historical_goalie_data, on="element_code", how="inner")
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)
NaN_master_historical_goalie_data = master_historical_goalie_data.fillna(0)
NaN_master_historical_goalie_data.sort_values('element_code').head()

goalie_hist_model_ready_df = NaN_master_historical_goalie_data
goalie_hist_model_ready_df.to_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/data/processed/goalie_hist_model_ready.csv")
goalie_hist_model_ready_df.columns

In [83]:
# I want a CLEAN csv of just goalie names, codes/ids, teams, and position
goalie_name_id_team_position = goalie_hist_model_ready_df.drop(columns = ['season', 'start_cost', 'end_cost', 'total_points', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
                                                               'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
                                                               'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
                                                               'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
                                                               'expected_goal_involvements', 'expected_goals_conceded'])
goalie_name_id_team_position.head()
goalie_name_id_team_position.to_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/data/processed/goalie_name_id_team_position.csv")

In [85]:
#Clean the goalie_hist_model_ready_df dataframe above

goalie_hist_model_ready_df[['id', 'code', 'element_code', 'first_name', 'second_name', 'team',
                            'element_type', 'season', 'start_cost', 'end_cost', 'total_points',
                            'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
                            'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
                            'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
                            'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
                            'expected_goal_involvements', 'expected_goals_conceded']]

goalie_hist_model_ready_df.head(10)

,id,first_name,second_name,team,element_type,code,element_code,season,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,5,Karl,Hein,1,1,463748,463748,2021,40,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00
1,5,Karl,Hein,1,1,463748,463748,2022,40,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00
2,5,Karl,Hein,1,1,463748,463748,2023,40,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00
3,15,David,Raya Martin,1,1,154561,154561,2021,45,44,95,2160,0,0,8,27,0,0,0,1,0,78,5,496,593.4,10.0,0,60.1,0,0.00,0.00,0.00,0.00
4,15,David,Raya Martin,1,1,154561,154561,2022,45,48,166,3420,0,0,12,46,1,0,0,1,0,154,20,822,1146.0,20.1,4,117.3,38,0.11,0.12,0.23,50.12
5,15,David,Raya Martin,1,1,154561,154561,2023,50,53,135,2880,0,0,16,24,0,1,0,2,0,46,6,569,413.0,0.0,0,41.1,32,0.00,0.04,0.04,22.51
6,73,Norberto,Murara Neto,1,1,69752,69752,2022,45,45,93,2385,0,0,6,39,0,0,0,4,0,100,9,510,727.6,0.0,0,72.8,27,0.00,0.02,0.02,44.70
7,73,Norberto,Murara Neto,1,1,69752,69752,2023,45,46,110,2880,0,0,7,55,0,1,0,5,0,113,14,592,875.2,0.0,0,87.5,32,0.00,0.01,0.01,49.66
8,39,Joe,Gauci,2,1,462492,462492,2023,40,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00
9,46,Filip,Marschall,2,1,462381,462381,2021,40,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00
